<a href="https://colab.research.google.com/github/retuyu88/digitalentkominfo/blob/master/Copy_of_%5B32_4%5D_CycleGAN_ADF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src = "https://i.imgur.com/UjutVJd.jpg" align = "center">

# CycleGAN Models 

This notebook demonstrates unpaired image to image translation using conditional GAN's, as described in [Unpaired Image-to-Image Translation using Cycle-Consistent Adversarial Networks](https://arxiv.org/abs/1703.10593), also known as CycleGAN. The paper proposes a method that can capture the characteristics of one image domain and figure out how these characteristics could be translated into another image domain, all in the absence of any paired training examples. 

This notebook assumes you are familiar with Pix2Pix, which you can learn about in the [Pix2Pix tutorial](https://www.tensorflow.org/beta/tutorials/generative/pix2pix). The code for CycleGAN is similar, the main difference is an additional loss function, and the use of unpaired training data.

CycleGAN uses a cycle consistency loss to enable training without the need for paired data. In other words, it can translate from one domain to another without a one-to-one mapping between the source and target domain. 

This opens up the possibility to do a lot of interesting tasks like photo-enhancement, image colorization, style transfer, etc. All you need is the source and the target dataset (which is simply a directory of images).

![Output Image 1](https://www.tensorflow.org/beta/tutorials/generative/images/horse2zebra_1.png)
![Output Image 2](https://www.tensorflow.org/beta/tutorials/generative/images/horse2zebra_2.png)

## Import Module

In [0]:

from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate
from tensorflow.keras.layers import BatchNormalization, Activation, ZeroPadding2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from PIL import Image

import datetime
import matplotlib.pyplot as plt
import sys
import numpy as np
import os

### Load Data Loader Helper Functions

In [0]:
!wget 'https://raw.githubusercontent.com/adf-telkomuniv/FG_AI_Public/master/others/cyclegan_data_loader.py'

from cyclegan_data_loader import DataLoader


In [0]:
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import imageio
from PIL import Image
import cv2 as cv


class DataLoader():
  def __init__(self, dataset_name, img_res=(128, 128)):
    self.dataset_name = dataset_name
    self.img_res = img_res
    
    
  def load_data(self, domain, batch_size=1, is_testing=False):
    data_type = "train%s" % domain if not is_testing else "test%s" % domain
    path = glob('./datasets/%s/%s/*' % (self.dataset_name, data_type))
    
    batch_images = np.random.choice(path, size=batch_size)
    
    imgs = []
    for img_path in batch_images:
        img = self.imread(img_path)
        if not is_testing:
          img = cv.resize(img, self.img_res, interpolation=cv.INTER_AREA)

          if np.random.random() > 0.5:
            img = np.fliplr(img)
        else:
          img = cv.resize(img, self.img_res, interpolation=cv.INTER_AREA)
        imgs.append(img)

    imgs = np.array(imgs)/127.5 - 1.

    return imgs

  def load_batch(self, batch_size=1, is_testing=False):
      data_type = "train" if not is_testing else "val"
      path_A = glob('./datasets/%s/%sA/*' % (self.dataset_name, data_type))
      path_B = glob('./datasets/%s/%sB/*' % (self.dataset_name, data_type))

      self.n_batches = int(min(len(path_A), len(path_B)) / batch_size)
      total_samples = self.n_batches * batch_size

      # Sample n_batches * batch_size from each path list so that model sees all
      # samples from both domains
      path_A = np.random.choice(path_A, total_samples, replace=False)
      path_B = np.random.choice(path_B, total_samples, replace=False)

      for i in range(self.n_batches-1):
          batch_A = path_A[i*batch_size:(i+1)*batch_size]
          batch_B = path_B[i*batch_size:(i+1)*batch_size]
          imgs_A, imgs_B = [], []
          for img_A, img_B in zip(batch_A, batch_B):
              img_A = self.imread(img_A)
              img_B = self.imread(img_B)

              img_A = cv.resize(img_A, self.img_res, interpolation=cv.INTER_AREA)
              img_B = cv.resize(img_B, self.img_res, interpolation=cv.INTER_AREA)

              if not is_testing and np.random.random() > 0.5:
                img_A = np.fliplr(img_A)
                img_B = np.fliplr(img_B)

              imgs_A.append(img_A)
              imgs_B.append(img_B)

          imgs_A = np.array(imgs_A)/127.5 - 1.
          imgs_B = np.array(imgs_B)/127.5 - 1.

          yield imgs_A, imgs_B

  def load_img(self, path):
      img = self.imread(path)
      img = cv.resize(img, self.img_res, interpolation=cv.INTER_AREA)
      img = img/127.5 - 1.
      return img[np.newaxis, :, :, :]

  def imread(self, path):
    return imageio.imread(path).astype(np.float)

---
## Load Dataset

Choose one dataset

### (A) Apple-to-Orange Dataset

In [0]:
!mkdir datasets

!wget -N 'https://people.eecs.berkeley.edu/~taesung_park/CycleGAN/datasets/apple2orange.zip' -O './datasets/apple2orange.zip'
!mkdir './datasets/apple2orange/'
!unzip -q './datasets/apple2orange.zip' -d ./datasets/
!rm './datasets/apple2orange.zip'
dataset_name = 'apple2orange'

### (B) Horse-to-Zebra Dataset

In [0]:
# !mkdir datasets

# !wget -N 'https://people.eecs.berkeley.edu/~taesung_park/CycleGAN/datasets/horse2zebra.zip' -O './datasets/horse2zebra.zip'
# !mkdir './datasets/horse2zebra/'
# !unzip -q './datasets/horse2zebra.zip' -d ./datasets/
# !rm './datasets/horse2zebra.zip'
# dataset_name = 'horse2zebra'

### Show Example

In [0]:
inp = Image.open('./datasets/apple2orange/trainA/n07740461_10012.jpg')
inp = np.array(inp).astype('float')
re = Image.open('./datasets/apple2orange/trainB/n07749192_1008.jpg')
re = np.array(re).astype('float')
# casting to int for matplotlib to show the image

plt.figure()
plt.imshow(inp/255.0)
plt.axis('off')
plt.figure()
plt.imshow(re/255.0)
plt.axis('off')

---
## Build CycleGAN Model

In [0]:
img_shape = (128, 128, 3)
data_loader = DataLoader(dataset_name=dataset_name,
                         img_res=(img_shape[0], img_shape[1]))

# Calculate output shape of D (PatchGAN)
patch = int(img_shape[0] / 2**4)
disc_patch = (patch, patch, 1)

# Number of filters in the first layer of G and D
gf = 32
df = 64

# Loss weights
lambda_cycle = 10.0                    # Cycle-consistency loss
lambda_id = 0.1 * lambda_cycle    # Identity loss

optimizer = Adam(0.0002, 0.5)

### Discriminator Model


In [0]:

def build_discriminator():

    def d_layer(layer_input, filters, f_size=4, normalization=True):
        """Discriminator layer"""
        d = Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
        d = LeakyReLU(alpha=0.2)(d)
        if normalization:
            d = BatchNormalization(renorm=True)(d)
        return d

    img = Input(shape=img_shape)

    d1 = d_layer(img, df, normalization=False)
    d2 = d_layer(d1, df*2)
    d3 = d_layer(d2, df*4)
    d4 = d_layer(d3, df*8)

    validity = Conv2D(1, kernel_size=4, strides=1, padding='same')(d4)

    return Model(img, validity)


In [0]:
# Build and compile the discriminators
d_A = build_discriminator()
d_B = build_discriminator()

d_A.compile(loss='mse', optimizer=optimizer, metrics=['accuracy'])
d_B.compile(loss='mse', optimizer=optimizer, metrics=['accuracy'])

### Generator Model


In [0]:

def build_generator():
    """U-Net Generator"""

    def conv2d(layer_input, filters, f_size=4):
        """Layers used during downsampling"""
        d = Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
        d = LeakyReLU(alpha=0.2)(d)
        d = BatchNormalization(renorm=True)(d)
        return d

    def deconv2d(layer_input, skip_input, filters, f_size=4, dropout_rate=0):
        """Layers used during upsampling"""
        u = UpSampling2D(size=2)(layer_input)
        u = Conv2D(filters, kernel_size=f_size, strides=1, padding='same', activation='relu')(u)
        if dropout_rate:
            u = Dropout(dropout_rate)(u)
        u = BatchNormalization(renorm=True)(u)
        u = Concatenate()([u, skip_input])
        return u

    # Image input
    d0 = Input(shape=img_shape)

    # Downsampling
    d1 = conv2d(d0, gf)
    d2 = conv2d(d1, gf*2)
    d3 = conv2d(d2, gf*4)
    d4 = conv2d(d3, gf*8)

    # Upsampling
    u1 = deconv2d(d4, d3, gf*4)
    u2 = deconv2d(u1, d2, gf*2)
    u3 = deconv2d(u2, d1, gf)

    u4 = UpSampling2D(size=2)(u3)
    output_img = Conv2D(3, kernel_size=4, strides=1, padding='same', activation='tanh')(u4)

    return Model(d0, output_img)

In [0]:
# Build the generators
g_AB = build_generator()
g_BA = build_generator()

### Combine Models

In [0]:
# Input images from both domains
img_A = Input(shape=img_shape)
img_B = Input(shape=img_shape)

# Translate images to the other domain
fake_B = g_AB(img_A)
fake_A = g_BA(img_B)
# Translate images back to original domain
reconstr_A = g_BA(fake_B)
reconstr_B = g_AB(fake_A)
# Identity mapping of images
img_A_id = g_BA(img_A)
img_B_id = g_AB(img_B)

# For the combined model we will only train the generators
d_A.trainable = False
d_B.trainable = False

# Discriminators determines validity of translated images
valid_A = d_A(fake_A)
valid_B = d_B(fake_B)

In [0]:
# Combined model trains generators to fool discriminators
combined = Model(inputs=[img_A, img_B],
                      outputs=[ valid_A, valid_B,
                                reconstr_A, reconstr_B,
                                img_A_id, img_B_id ])
combined.compile(loss=['mse', 'mse',
                            'mae', 'mae',
                            'mae', 'mae'],
                    loss_weights=[  1, 1,
                                    lambda_cycle, lambda_cycle,
                                    lambda_id, lambda_id ],
                    optimizer=optimizer)

---
## Training Model

### Sampling Helper Function

In [0]:
def sample_images(epoch, batch_i):
    os.makedirs('images/%s' % dataset_name, exist_ok=True)
    r, c = 2, 3

    imgs_A = data_loader.load_data(domain="A", batch_size=1, is_testing=True)
    imgs_B = data_loader.load_data(domain="B", batch_size=1, is_testing=True)

    # Demo (for GIF)
    #imgs_A = self.data_loader.load_img('datasets/apple2orange/testA/n07740461_1541.jpg')
    #imgs_B = self.data_loader.load_img('datasets/apple2orange/testB/n07749192_4241.jpg')

    # Translate images to the other domain
    fake_B = g_AB.predict(imgs_A)
    fake_A = g_BA.predict(imgs_B)
    
    # Translate back to original domain
    reconstr_A = g_BA.predict(fake_B)
    reconstr_B = g_AB.predict(fake_A)

    gen_imgs = np.concatenate([imgs_A, fake_B, reconstr_A, imgs_B, fake_A, reconstr_B])

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    titles = ['Original', 'Translated', 'Reconstructed']
    fig, axs = plt.subplots(r, c, figsize=(6,6))
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt])
            axs[i, j].set_title(titles[j])
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("images/%s/%d_%d.png" % (dataset_name, epoch, batch_i))
    plt.show()
    plt.close()


### Training Function

In [0]:
def train(epochs, batch_size=1, print_every=50):

    start_time = datetime.datetime.now()

    # Adversarial loss ground truths
    valid = np.ones((batch_size,) + disc_patch)
    fake = np.zeros((batch_size,) + disc_patch)

    for epoch in range(epochs):
        for batch_i, (imgs_A, imgs_B) in enumerate(data_loader.load_batch(batch_size)):

            # ----------------------
            #  Train Discriminators
            # ----------------------

            # Translate images to opposite domain
            fake_B = g_AB.predict(imgs_A)
            fake_A = g_BA.predict(imgs_B)

            # Train the discriminators (original images = real / translated = Fake)
            dA_loss_real = d_A.train_on_batch(imgs_A, valid)
            dA_loss_fake = d_A.train_on_batch(fake_A, fake)
            dA_loss = 0.5 * np.add(dA_loss_real, dA_loss_fake)

            dB_loss_real = d_B.train_on_batch(imgs_B, valid)
            dB_loss_fake = d_B.train_on_batch(fake_B, fake)
            dB_loss = 0.5 * np.add(dB_loss_real, dB_loss_fake)

            # Total disciminator loss
            d_loss = 0.5 * np.add(dA_loss, dB_loss)


            # ------------------
            #  Train Generators
            # ------------------

            # Train the generators
            g_loss = combined.train_on_batch([imgs_A, imgs_B],
                                                    [valid, valid,
                                                    imgs_A, imgs_B,
                                                    imgs_A, imgs_B])


            # If at save interval => save generated image samples
            if batch_i % print_every == 0:
                sample_images(epoch, batch_i)
                elapsed_time = datetime.datetime.now() - start_time

                # Plot the progress
                print ("[Epoch %d/%d] [Batch %d/%d] [D loss: %f, acc: %3d%%] [G loss: %05f, adv: %05f, recon: %05f, id: %05f] time: %s " \
                       % ( epoch, epochs, batch_i, data_loader.n_batches, d_loss[0], 100*d_loss[1], g_loss[0], np.mean(g_loss[1:3]),
                          np.mean(g_loss[3:5]), np.mean(g_loss[5:6]), elapsed_time))


### Train the Model

In [0]:
train(epochs=50, batch_size=32, print_every=100)

---
## Testing Model

In [0]:
r, c = 2, 3

imgs_A = data_loader.load_data(domain="A", batch_size=1, is_testing=True)
imgs_B = data_loader.load_data(domain="B", batch_size=1, is_testing=True)

# Demo (for GIF)
#imgs_A = self.data_loader.load_img('datasets/apple2orange/testA/n07740461_1541.jpg')
#imgs_B = self.data_loader.load_img('datasets/apple2orange/testB/n07749192_4241.jpg')

# Translate images to the other domain
fake_B = g_AB.predict(imgs_A)
fake_A = g_BA.predict(imgs_B)

# Translate back to original domain
reconstr_A = g_BA.predict(fake_B)
reconstr_B = g_AB.predict(fake_A)

gen_imgs = np.concatenate([imgs_A, fake_B, reconstr_A, imgs_B, fake_A, reconstr_B])

# Rescale images 0 - 1
gen_imgs = 0.5 * gen_imgs + 0.5

titles = ['Original', 'Translated', 'Reconstructed']
fig, axs = plt.subplots(r, c, figsize=(10,10))
cnt = 0
for i in range(r):
    for j in range(c):
        axs[i,j].imshow(gen_imgs[cnt])
        axs[i, j].set_title(titles[j])
        axs[i,j].axis('off')
        cnt += 1
plt.show()
plt.close()

<p>Copyright &copy; 2019 <a href=https://www.linkedin.com/in/andityaarifianto/>ADF</a> </p>